# LSTM

### Importing packages

In [1]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
#https://www.liip.ch/en/blog/sentiment-detection-with-keras-word-embeddings-and-lstm-deep-learning-networks

# LSTM for sequence classification in the IMDB dataset
import numpy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import text,sequence
from tensorflow.keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten, Reshape
from tensorflow.keras.layers import concatenate, GRU, Input, CuDNNLSTM, MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.regularizers import l2
from tensorflow.python.keras import backend as k

import tensorflow.keras
from tensorflow.keras import optimizers
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.layers import BatchNormalization
from time import time
from tensorflow.python.keras.callbacks import TensorBoard

# fix random seed for reproducibility
numpy.random.seed(7)

import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


### Loading Data

In [2]:
df = pd.read_csv("train_data.csv")

In [3]:
df.head(3)

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,Mrs.,IN,2016-12-05 13:43:57,Grades PreK-2,Literacy & Language,"ESL, Literacy",Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,0
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,Mr.,FL,2016-10-25 09:22:10,Grades 6-8,"History & Civics, Health & Sports","Civics & Government, Team Sports",Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,1
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,Ms.,AZ,2016-08-31 12:03:56,Grades 6-8,Health & Sports,"Health & Wellness, Team Sports",Soccer Equipment for AWESOME Middle School Stu...,"\r\n\""True champions aren't always the ones th...",The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0


#### 'Resource_summary_has_digit'

In [4]:
import re
def hasNumbers(inputString):
    return bool(re.search(r'\d',inputString))
 
resource_summary=list(df['project_resource_summary'].values)
has_digits = [] 
for i in resource_summary:
    if (hasNumbers(i)==True):
        has_digits.append(1)
    else:        
        has_digits.append(0)

#### 'project_title'

In [5]:
# https://stackoverflow.com/a/47091490/4084039
import re


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [6]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]


from tqdm import tqdm
preprocessed_title = []
# tqdm is for printing the status bar
for sentance in tqdm(df['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('nannan', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = sent.lower()
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split(" ") if e not in stopwords)
    preprocessed_title.append(sent.lower().strip())

100%|███████████████████████████████████████████████████████████████████████| 109248/109248 [00:02<00:00, 51669.89it/s]


#### 'project_resource_summary'

In [7]:
from tqdm import tqdm
summary = []
# tqdm is for printing the status bar
for sentance in tqdm(df['project_resource_summary'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('nannan', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = sent.lower()
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split(" ") if e not in stopwords)
    summary.append(sent.lower().strip())

100%|███████████████████████████████████████████████████████████████████████| 109248/109248 [00:04<00:00, 22683.49it/s]


In [8]:
project_data = pd.read_csv("preprocessed_data.csv")

In [9]:
project_data["title"] = preprocessed_title

In [10]:
project_data['summary'] = summary

In [11]:
project_data['summary_digits']=has_digits

In [12]:
project_data.shape

(109248, 12)

In [13]:
project_data.head()

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,title,summary,summary_digits
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05,educational support english learners home,students need opportunities practice beginning...,0
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03,wanted projector hungry learners,students need projector help viewing education...,0
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00,soccer equipment awesome middle school students,students need shine guards athletic socks socc...,0
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04,techie kindergarteners,students need engage reading math way inspire ...,0
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74,interactive math tools,students need hands practice mathematics fun p...,0


In [14]:
#Printing the attributes of project_data
print("Attributes :", project_data.columns.values)

Attributes : ['school_state' 'teacher_prefix' 'project_grade_category'
 'teacher_number_of_previously_posted_projects' 'project_is_approved'
 'clean_categories' 'clean_subcategories' 'essay' 'price' 'title'
 'summary' 'summary_digits']


### Resource data

In [15]:
resource_data = pd.read_csv('resources.csv')

In [16]:
resource_data.head(3)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45


In [17]:
# reference : https://stackoverflow.com/questions/22407798/how-to-reset-a-dataframes-indexes-for-all-groups-in-one-step
price_data = resource_data.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()
price_data.head(2)

,id,price,quantity
0,p000001,459.56,7
1,p000002,515.89,21


In [18]:
# join two dataframes(project_data and price_data) in python
# reference : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
project_data['price'] = resource_data['price']
project_data['quantity'] = resource_data['quantity']

In [19]:
project_data.head(2)

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,title,summary,summary_digits,quantity
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,149.00,educational support english learners home,students need opportunities practice beginning...,0,1
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,14.95,wanted projector hungry learners,students need projector help viewing education...,0,3


#### 'Total numerical features'

In [20]:
#numerical inputs
project_data['num'] = project_data['teacher_number_of_previously_posted_projects'] + project_data['price'] + project_data['quantity']+project_data['summary_digits']

#### 'Total text data'

In [21]:
project_data['total_text'] = project_data['essay']+ " " +project_data['title']+ " " + project_data['summary']

In [22]:
project_data.head()

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price,title,summary,summary_digits,quantity,num,total_text
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,149.00,educational support english learners home,students need opportunities practice beginning...,0,1,203.00,i fortunate enough use fairy tale stem kits cl...
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,14.95,wanted projector hungry learners,students need projector help viewing education...,0,3,21.95,imagine 8 9 years old you third grade classroo...
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,8.45,soccer equipment awesome middle school students,students need shine guards athletic socks socc...,0,1,19.45,having class 24 students comes diverse learner...
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,13.59,techie kindergarteners,students need engage reading math way inspire ...,0,2,17.59,i recently read article giving students choice...
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,24.95,interactive math tools,students need hands practice mathematics fun p...,0,3,29.95,my students crave challenge eat obstacles brea...


In [23]:
col = ['teacher_number_of_previously_posted_projects', 'price', 'quantity','essay','title','summary','summary_digits']

project_data.drop(labels=col,axis =1, inplace=True)

### Final dataset

In [24]:
project_data.head()

,school_state,teacher_prefix,project_grade_category,project_is_approved,clean_categories,clean_subcategories,num,total_text
0,ca,mrs,grades_prek_2,1,math_science,appliedsciences health_lifescience,203.00,i fortunate enough use fairy tale stem kits cl...
1,ut,ms,grades_3_5,1,specialneeds,specialneeds,21.95,imagine 8 9 years old you third grade classroo...
2,ca,mrs,grades_prek_2,1,literacy_language,literacy,19.45,having class 24 students comes diverse learner...
3,ga,mrs,grades_prek_2,1,appliedlearning,earlydevelopment,17.59,i recently read article giving students choice...
4,wa,mrs,grades_3_5,1,literacy_language,literacy,29.95,my students crave challenge eat obstacles brea...


In [25]:
y = project_data['project_is_approved']
project_data.drop(['project_is_approved'], axis=1, inplace=True)
X = project_data

print(X.shape)
print(y.shape)

(109248, 7)
(109248,)


## Preparing Data for model

#### Splitting data, stratify sampling

In [26]:
#splitting data
X_train,X_test,y_train,y_test = train_test_split(X, y , stratify = y, train_size = 0.7)
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,stratify = y_train,train_size = 0.7)

In [27]:
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

(53531, 7) (53531,)
(22942, 7) (22942,)
(32775, 7) (32775,)


#### Label to categorical

In [28]:
#converting class labels to categorical variables
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_cv = to_categorical(y_cv)

#### Featurization and padding text data

In [36]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(X_train['total_text'])
word_index = t.word_index
text_train = t.texts_to_sequences(X_train['total_text'])
text_test = t.texts_to_sequences(X_test['total_text'])
text_cv = t.texts_to_sequences(X_cv['total_text'])


from keras.preprocessing.sequence import pad_sequences
max_review_length = 300
text_train = pad_sequences(text_train, maxlen=max_review_length) 
text_test = pad_sequences(text_test, maxlen=max_review_length)
text_cv = pad_sequences(text_cv, maxlen=max_review_length)

In [37]:
text_train[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   10, 1572,   70, 2212,
       1567,    7,    1,    5, 1572, 1008, 1541,    7,    1,  621, 1767,
        114,   14,  947, 2425,  279,  327,  264, 2425,  279,  197,  456,
          3,   31,  162,  177,   38,   39,   89,    2,   50,  125,   38,
        284,    3,  524,  455, 4935,    1,  657,   72,  173,   92,   14,
         25,   24, 1564,  326,    7,    1,   19,   24,  484,  968,  603,
        649, 2767,    7,    3, 2505,   24,  215,   

#### Creating embedding matrix using pretrain golve model

In [29]:
embeddings_index = {}
f = open('glove.42B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917495 word vectors.


In [38]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [39]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15243   , -0.16945   , -0.022748  , ...,  0.61800998,
         0.41281   ,  0.0010077 ],
       [-0.043504  , -0.18483999, -0.14613   , ...,  0.1008    ,
         0.1068    ,  0.089065  ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14264999, -0.20883   ,  0.53634   , ...,  0.19422001,
         0.062518  ,  0.018873  ]])

### Tokenizing categorical data

#### 1. School_state

In [40]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(X_train['school_state'])

state_train = le.transform(X_train['school_state'])
state_test = le.transform(X_test['school_state'])
state_cv = le.transform(X_cv['school_state'])

print(state_train.shape)

(53531,)


#### 2.Teacher_prefix

In [41]:
le = preprocessing.LabelEncoder()
le.fit(X_train['teacher_prefix'])

prefix_train = le.transform(X_train['teacher_prefix'])
prefix_test = le.transform(X_test['teacher_prefix'])
prefix_cv = le.transform(X_cv['teacher_prefix'])

print(prefix_train.shape)

(53531,)


#### 3. Project_grade_category

In [42]:
le = preprocessing.LabelEncoder()
le.fit(X_train['project_grade_category'])

grade_train = le.transform(X_train['project_grade_category'])
grade_test = le.transform(X_test['project_grade_category'])
grade_cv = le.transform(X_cv['project_grade_category'])

print(grade_train.shape)

(53531,)


#### 4. Clean_categories

In [43]:
t = Tokenizer()
t.fit_on_texts(X_train['clean_categories'])
word_index = t.word_index
clean_cat_train = t.texts_to_sequences(X_train['clean_categories'])
clean_cat_test = t.texts_to_sequences(X_test['clean_categories'])
clean_cat_cv = t.texts_to_sequences(X_cv['clean_categories'])

from keras.preprocessing.sequence import pad_sequences
max_review_length = 5
clean_cat_train = pad_sequences(clean_cat_train, maxlen=max_review_length) 
clean_cat_test = pad_sequences(clean_cat_test, maxlen=max_review_length)
clean_cat_cv = pad_sequences(clean_cat_cv, maxlen=max_review_length)

print(clean_cat_train.shape)

(53531, 5)


#### 5. Clean_subcategories

In [44]:
t = Tokenizer()
t.fit_on_texts(X_train['clean_subcategories'])
word_index = t.word_index
clean_subcat_train = t.texts_to_sequences(X_train['clean_subcategories'])
clean_subcat_test = t.texts_to_sequences(X_test['clean_subcategories'])
clean_subcat_cv = t.texts_to_sequences(X_cv['clean_subcategories'])

from keras.preprocessing.sequence import pad_sequences
max_review_length = 5
clean_subcat_train = pad_sequences(clean_subcat_train, maxlen=max_review_length) 
clean_subcat_test = pad_sequences(clean_subcat_test, maxlen=max_review_length)
clean_subcat_cv = pad_sequences(clean_subcat_cv, maxlen=max_review_length)

print(clean_subcat_train.shape)

(53531, 5)


### Numerical feature

In [45]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(X_train['num'].values.reshape(-1,1))
# finding the mean and standar 
 
# Now standardize the data with above maen and variance. 
num_train = scalar.transform(X_train['num'].values.reshape(-1, 1))
num_cv = scalar.transform(X_cv['num'].values.reshape(-1, 1))
num_test = scalar.transform(X_test['num'].values.reshape(-1, 1))

print(num_train.shape)

(53531, 1)


### Function for AUC Score

In [46]:
#https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
#https://developpaper.com/question/how-to-apply-the-custom-operation-of-py_func-in-tensorflow-to-keras/
def auc(y_true, y_pred) :
    score = tf.py_func( lambda y_true, y_pred : roc_auc_score(y_true, y_pred, average='macro', sample_weight=None).astype('float32'),
                        [y_true, y_pred],
                        'float32',
                        stateful=True,
                        name='sklearnAUC')
    return score

# LSTM Model 1

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

### To find the embedding size and vocab

In [47]:
no_of_unique_cat  = X_train['school_state'].nunique()
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
print("School_state")
print("embedding_output_dim",int(embedding_size))
print('vocab', no_of_unique_cat+1)
print("-"*50)

no_of_unique_cat  = X_train['teacher_prefix'].nunique()
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
print("Teacher_prefix")
print("embedding_output_dim",int(embedding_size))
print('vocab', no_of_unique_cat+1)
print("-"*50)

no_of_unique_cat  = X_train['project_grade_category'].nunique()
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
print("Grade_category")
print("embedding_output_dim",int(embedding_size))
print('vocab', no_of_unique_cat+1)
print("-"*50)

no_of_unique_cat  = X_train['clean_categories'].nunique()
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
print("Subject_categories")
print("embedding_output_dim",int(embedding_size))
print('vocab', no_of_unique_cat+1)
print("-"*50)

no_of_unique_cat  = X_train['clean_subcategories'].nunique()
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
print("Subject_categories")
print("embedding_output_dim",int(embedding_size))
print('vocab', no_of_unique_cat+1)

School_state
embedding_output_dim 26
vocab 52
--------------------------------------------------
Teacher_prefix
embedding_output_dim 3
vocab 6
--------------------------------------------------
Grade_category
embedding_output_dim 2
vocab 5
--------------------------------------------------
Subject_categories
embedding_output_dim 25
vocab 51
--------------------------------------------------
Subject_categories
embedding_output_dim 50
vocab 376


In [48]:
tf.keras.backend.clear_session()

#Essay input --> 1
text = Input(shape=(300,), name="text")
x1 = Embedding(input_dim=46454,output_dim=300,trainable=False,weights=[embedding_matrix])(text)
x1 = SpatialDropout1D(0.3)(x1)
x1 = CuDNNLSTM(256,return_sequences=True)(x1)
x1 = Flatten()(x1)


#State input --> 2
state = Input(shape=(1,), name="state")
x2 = Embedding(input_dim=52,output_dim=26)(state)
x2 = Flatten()(x2)

#Teacher prefix input --> 3
prefix = Input(shape=(1,), name="prefix")
x3 = Embedding(input_dim=6,output_dim=3)(prefix)
x3 = Flatten()(x3)

#Grade category input --> 4
grade = Input(shape=(1,), name="grade")
x4 = Embedding(input_dim=5,output_dim=2)(grade)
x4 = Flatten()(x4)

#Subject category input --> 5
subj_cat = Input(shape=(5,), name="subject_category")
x5 = Embedding(input_dim=51,output_dim=25)(subj_cat)
x5 = Flatten()(x5)

#Subject subcategory input --> 6
subj_subcat = Input(shape=(5,), name="subject_sub_category")
x6 = Embedding(input_dim=376,output_dim=50)(subj_subcat)
x6 = Flatten()(x6)


#Numerical input -->7
num = Input(shape=(1,), name="numerical")
x7 = (Dense(16, activation='relu'))(num)



concat = concatenate([x1,x2,x3,x4,x5,x6,x7])

x = Dense(128,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(concat)
x = Dropout(0.4)(x)
x = Dense(64,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(32,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)

output = Dense(2, activation = 'softmax')(x)
 

model = Model([text,state,prefix,grade,subj_cat,subj_subcat,num], output)

#https://www.youtube.com/watch?v=2U6Jl7oqRkM
#Instantiating tensorboard for model visualization
#To visualize, run -  tensorboard --log_dir=logs/{} in command prompt
tensorboard = TensorBoard(log_dir="logs/".format(time))

model.compile(loss="categorical_crossentropy", optimizer= tensorflow.keras.optimizers.Adam(lr=0.0006,decay = 1e-4), metrics=[auc])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Model: "model"
__________________________________________________________________________________________________
Layer (type)  

In [49]:
filepath="weights_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')

model.fit([text_train,state_train,grade_train,prefix_train,clean_cat_train,clean_subcat_train,num_train], y_train, epochs=15, verbose=1, batch_size=300, validation_data=([text_cv,state_cv,grade_cv,prefix_cv,clean_cat_cv,clean_subcat_cv,num_cv], y_cv), callbacks=[tensorboard,checkpoint])

Train on 53531 samples, validate on 22942 samples
Epoch 1/15
53400/53531 [============================>.] - ETA: 0s - loss: 0.5248 - auc: 0.5486
Epoch 00001: val_auc improved from -inf to 0.39083, saving model to weights_1.hdf5
53531/53531 [==============================] - 100s 2ms/sample - loss: 0.5247 - auc: 0.5488 - val_loss: 0.5428 - val_auc: 0.3908
Epoch 2/15
53400/53531 [============================>.] - ETA: 0s - loss: 0.4518 - auc: 0.6576  E - ETA: 16s - lo - ETA: 14s - loss: 0.4583 - auc:  - ETA: 13s - loss: 0.4591 - a
Epoch 00002: val_auc improved from 0.39083 to 0.70053, saving model to weights_1.hdf5
53531/53531 [==============================] - 43s 812us/sample - loss: 0.4518 - auc: 0.6571 - val_loss: 0.4320 - val_auc: 0.7005
Epoch 3/15
53400/53531 [============================>.] - ETA: 0s - loss: 0.4320 - auc: 0.6872
Epoch 00003: val_auc improved from 0.70053 to 0.71539, saving model to weights_1.hdf5
53531/53531 [==============================] - 43s 805us/sample - lo

#### compiling model weights

In [50]:
model.load_weights("weights_1.hdf5")
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[auc])

#### Model visualization

In [51]:
print("Train AUC",roc_auc_score(y_train,(model.predict([text_train,state_train,grade_train,prefix_train,clean_cat_train,clean_subcat_train,num_train]))))
print("-"*50)
print("CV AUC",roc_auc_score(y_cv,(model.predict([text_cv,state_cv,grade_cv,prefix_cv,clean_cat_cv,clean_subcat_cv,num_cv]))))
print("-"*50)
print("Test AUC",roc_auc_score(y_test,(model.predict([text_test,state_test,grade_test,prefix_test,clean_cat_test,clean_subcat_test,num_test]))))

Train AUC 0.7864668722630237
--------------------------------------------------
CV AUC 0.7450655336380656
--------------------------------------------------
Test AUC 0.760104170038495


In [52]:
"""# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs --port=8008"""

'# Load TENSORBOARD\n%load_ext tensorboard\n# Start TENSORBOARD\n%tensorboard --logdir logs --port=8008'

<img src="model1.png">

# --------------------------------------------  Model 2  ----------------------------------------------------

In [53]:
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

(53531, 7) (53531, 2)
(22942, 7) (22942, 2)
(32775, 7) (32775, 2)


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
tfidf = TfidfVectorizer()
data_text = tfidf.fit_transform(X_train['total_text'])
plt.boxplot(tfidf.idf_)
plt.ylabel("IDF score")

Text(0, 0.5, 'IDF score')

In [55]:
print("25 percentile (idf):", np.percentile(tfidf.idf_,[25]))
print("50 percentile (idf):",np.percentile(tfidf.idf_,[50]))
print("75 percentile (idf):",np.percentile(tfidf.idf_,[75]))
print("90 percentile (idf):",np.percentile(tfidf.idf_,[90]))
print("95 percentile (idf):",np.percentile(tfidf.idf_,[95]))
print("99 percentile (idf):",np.percentile(tfidf.idf_,[99]))

25 percentile (idf): [9.17998468]
50 percentile (idf): [10.7894226]
75 percentile (idf): [11.1948877]
90 percentile (idf): [11.1948877]
95 percentile (idf): [11.1948877]
99 percentile (idf): [11.1948877]


In [56]:
feature_idf = zip(tfidf.get_feature_names(),tfidf.idf_)

feature_name = []
for x,y in feature_idf:
    
    if y >=2 and 11.19:
        feature_name.append(x)
    else:
        pass

In [57]:
len(feature_name)

46385

### Considering only those features with idf value between 25th and 75th percentile in 'project_essay'

In [61]:
def reduced_text(df):
    processed_text = []
    for text in df:
        sent = " "
        words = text.split()
        for word in words:
            if word in feature_name:
                sent = sent +" " + word
            else:
                pass
            
        processed_text.append(sent)
    return processed_text

train_text_reduced = reduced_text(X_train['total_text'])
test_text_reduced = reduced_text(X_test['total_text'])
cv_text_reduced = reduced_text(X_cv['total_text'])

In [62]:
train_text_reduced[0]

'  privilege working amazing although majority low socioeconomic families bring curiosity eagerness everyday diverse backgrounds bring unique perspective individual education plans unfortunately sedentary structures create obstacles best frequent movement options traditional furniture meet needs stress unable move needed receiving instruction cause disengaged often end receiving disciplinary referrals acting movement needs requesting exercise ball chairs give alternative traditional chairs offered exercise ball pretty popular problem ball stability necessary produce quality project special needs sensory challenges experience without discomfort traditional seating these chairs give opportunity exercise ball way keep focused rolling away listening literature books cds well listening center utilize cds levels esl'

#### Featurization and padding text data

In [66]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(train_text_reduced)
word_index = t.word_index
text_train_reduced = t.texts_to_sequences(train_text_reduced)
text_test_reduced = t.texts_to_sequences(test_text_reduced)
text_cv_reduced = t.texts_to_sequences(cv_text_reduced)


from keras.preprocessing.sequence import pad_sequences
max_review_length = 200
text_train_reduced = pad_sequences(text_train_reduced, maxlen=max_review_length) 
text_test_reduced = pad_sequences(text_test_reduced, maxlen=max_review_length)
text_cv_reduced = pad_sequences(text_cv_reduced, maxlen=max_review_length)

In [67]:
text_train_reduced[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 1521,   48,  136,  559,  310,   63,  413,   84,  201,  811,
       1711,  189,   83,  122,  201,  261, 2314,  273,   27, 1414,  524,
       4220, 1578,   18,  839,   28, 2820,  185,  197,  527, 1205,  180,
         23, 1066,  949,   72,  183,  696,  159, 1770, 7098,  126,  416,
        696, 6428, 8904, 3588,  185,   23,  335,  4

#### Creating embedding matrix using pretrain golve model

In [68]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [69]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.50835001,  0.17623   , -0.16424   , ...,  0.77085   ,
         0.15795   , -0.080663  ],
       [-0.26078001, -0.36897999, -0.022831  , ...,  0.23384   ,
         0.24267   ,  0.091846  ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14264999, -0.20883   ,  0.53634   , ...,  0.19422001,
         0.062518  ,  0.018873  ]])

In [71]:
tf.keras.backend.clear_session()

#Essay input --> 1
text = Input(shape=(200,), name="text")
x1 = Embedding(input_dim=46386,output_dim=300,trainable=False,weights=[embedding_matrix])(text)
x1 = SpatialDropout1D(0.3)(x1)
x1 = CuDNNLSTM(256,return_sequences=True)(x1)
x1 = Flatten()(x1)


#State input --> 2
state = Input(shape=(1,), name="state")
x2 = Embedding(input_dim=52,output_dim=26)(state)
x2 = Flatten()(x2)

#Teacher prefix input --> 3
prefix = Input(shape=(1,), name="prefix")
x3 = Embedding(input_dim=6,output_dim=3)(prefix)
x3 = Flatten()(x3)

#Grade category input --> 4
grade = Input(shape=(1,), name="grade")
x4 = Embedding(input_dim=5,output_dim=2)(grade)
x4 = Flatten()(x4)

#Subject category input --> 5
subj_cat = Input(shape=(5,), name="subject_category")
x5 = Embedding(input_dim=51,output_dim=25)(subj_cat)
x5 = Flatten()(x5)

#Subject subcategory input --> 6
subj_subcat = Input(shape=(5,), name="subject_sub_category")
x6 = Embedding(input_dim=376,output_dim=50)(subj_subcat)
x6 = Flatten()(x6)


#Numerical input -->7
num = Input(shape=(1,), name="numerical")
x7 = (Dense(16, activation='relu'))(num)



concat = concatenate([x1,x2,x3,x4,x5,x6,x7])

x = Dense(128,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(concat)
x = Dropout(0.4)(x)
x = Dense(64,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(32,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)

output = Dense(2, activation = 'softmax')(x)
 

model = Model([text,state,prefix,grade,subj_cat,subj_subcat,num], output)

#https://www.youtube.com/watch?v=2U6Jl7oqRkM
#Instantiating tensorboard for model visualization
#To visualize, run -  tensorboard --log_dir=logs/{} in command prompt
tensorboard = TensorBoard(log_dir="logs/".format(time))

model.compile(loss="categorical_crossentropy", optimizer= tensorflow.keras.optimizers.Adam(lr=0.0006,decay = 1e-4), metrics=[auc])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 300)     13915800    text[0][0]                       
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 200, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
state (InputLayer)              [(None, 1)]          0                                            
______________________________________________________________________________________________

In [72]:
filepath="weights_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')

model.fit([text_train_reduced,state_train,grade_train,prefix_train,clean_cat_train,clean_subcat_train,num_train], y_train, epochs=15, verbose=1, batch_size=250, validation_data=([text_cv_reduced,state_cv,grade_cv,prefix_cv,clean_cat_cv,clean_subcat_cv,num_cv], y_cv), callbacks=[tensorboard,checkpoint])

Train on 53531 samples, validate on 22942 samples
Epoch 1/15
53500/53531 [============================>.] - ETA: 0s - loss: 0.5560 - auc: 0.5364
Epoch 00001: val_auc improved from -inf to 0.62902, saving model to weights_2.hdf5
53531/53531 [==============================] - 38s 714us/sample - loss: 0.5561 - auc: 0.5370 - val_loss: 0.4657 - val_auc: 0.6290
Epoch 2/15
53500/53531 [============================>.] - ETA: 0s - loss: 0.4785 - auc: 0.6116
Epoch 00002: val_auc improved from 0.62902 to 0.67708, saving model to weights_2.hdf5
53531/53531 [==============================] - 29s 537us/sample - loss: 0.4785 - auc: 0.6124 - val_loss: 0.4454 - val_auc: 0.6771
Epoch 3/15
53500/53531 [============================>.] - ETA: 0s - loss: 0.4515 - auc: 0.6698
Epoch 00003: val_auc improved from 0.67708 to 0.67890, saving model to weights_2.hdf5
53531/53531 [==============================] - 29s 536us/sample - loss: 0.4515 - auc: 0.6690 - val_loss: 0.4354 - val_auc: 0.6789
Epoch 4/15
53500/535

#### compiling model weights

In [73]:
model.load_weights("weights_2.hdf5")
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[auc])

#### Model visualization

In [74]:
print("Train AUC",roc_auc_score(y_train,(model.predict([text_train_reduced,state_train,grade_train,prefix_train,clean_cat_train,clean_subcat_train,num_train]))))
print("-"*50)
print("CV AUC",roc_auc_score(y_cv,(model.predict([text_cv_reduced,state_cv,grade_cv,prefix_cv,clean_cat_cv,clean_subcat_cv,num_cv]))))
print("-"*50)
print("Test AUC",roc_auc_score(y_test,(model.predict([text_test_reduced,state_test,grade_test,prefix_test,clean_cat_test,clean_subcat_test,num_test]))))

Train AUC 0.8240843559440708
--------------------------------------------------
CV AUC 0.7360593751179179
--------------------------------------------------
Test AUC 0.7463178622047653


In [77]:
"""# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs --port=8008"""

'# Load TENSORBOARD\n%load_ext tensorboard\n# Start TENSORBOARD\n%tensorboard --logdir logs --port=8008'

<img src="model2.png">

# --------------------------------------------  Model 3  ----------------------------------------------------

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(lowercase= False)
token = CountVectorizer()

school_state_train = (token.fit_transform(X_train['school_state'])).toarray()
school_state_test = (token.transform(X_test['school_state'])).toarray()
school_state_cv = (token.transform(X_cv['school_state'])).toarray()

print(school_state_train.shape)

(53531, 51)


In [36]:
prefix_train = token.fit_transform(X_train['teacher_prefix']).toarray()
prefix_cv = token.transform(X_cv['teacher_prefix']).toarray()
prefix_test = token.transform(X_test['teacher_prefix']).toarray()
print(prefix_train.shape)

(53531, 5)


In [37]:
grade_train = token.fit_transform(X_train['project_grade_category']).toarray()
grade_cv = token.transform(X_cv['project_grade_category']).toarray()
grade_test = token.transform(X_test['project_grade_category']).toarray()
print(grade_train.shape)

(53531, 4)


In [38]:
cat_train = token.fit_transform(X_train['clean_categories']).toarray()
cat_cv = token.transform(X_cv['clean_categories']).toarray()
cat_test = token.transform(X_test['clean_categories']).toarray()
print(cat_train.shape)

(53531, 9)


In [39]:
subcat_train = token.fit_transform(X_train['clean_subcategories']).toarray()
subcat_cv = token.transform(X_cv['clean_subcategories']).toarray()
subcat_test = token.transform(X_test['clean_subcategories']).toarray()
print(subcat_train.shape)

(53531, 30)


In [40]:
train_num = X_train['num'].values.reshape(-1,1)
cv_num = X_cv['num'].values.reshape(-1,1)
test_num = X_test['num'].values.reshape(-1,1)
print(train_num.shape)
print(cv_num.shape)
print(test_num.shape)

(53531, 1)
(22942, 1)
(32775, 1)


In [43]:
cat_num_train_feat = np.hstack((school_state_train,prefix_train,grade_train,cat_train,subcat_train,train_num))
cat_num_cv_feat = np.hstack((school_state_cv,prefix_cv,grade_cv,cat_cv,subcat_cv,cv_num))
cat_num_test_feat = np.hstack((school_state_test,prefix_test,grade_test,cat_test,subcat_test,test_num))
print(cat_num_train_feat.shape)
print(cat_num_cv_feat.shape)
print(cat_num_test_feat.shape)

(53531, 100)
(22942, 100)
(32775, 100)


In [44]:
cat_num_train_feat = np.resize(cat_num_train_feat, new_shape=(53531,100,1))
cat_num_cv_feat = np.resize(cat_num_cv_feat, new_shape=(22942,100,1))
cat_num_test_feat = np.resize(cat_num_test_feat, new_shape=(32775,100,1))

In [45]:
tf.keras.backend.clear_session()

# input 1
essay = Input(batch_shape=(None,300), name="essay_input")
x1 = Embedding(input_dim=46454,output_dim = 300,weights=[embedding_matrix],trainable = False)(essay)
x1 = SpatialDropout1D(0.4)(x1)
x1 = CuDNNLSTM(256,return_sequences=True)(x1)
x1 = Flatten()(x1)

# input 2
other = Input(shape=(100,1),name="other_input")
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(other)
x2 = BatchNormalization()(x2)
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(x2)
x2 = Flatten()(x2)


concat = concatenate([x1,x2])


x = Dense(64,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(concat)
x = Dropout(0.3)(x)
x = Dense(64,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(126,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)

output = Dense(2, activation = 'softmax')(x)
 
model = Model([essay,other], output)

#To visualize, run -  tensorboard --log_dir=logs/ in command prompt

tensorboard = TensorBoard(log_dir="logs/".format(time))

model.compile(loss="categorical_crossentropy", optimizer= tensorflow.keras.optimizers.Adam(lr=0.0006,decay = 1e-4), metrics=[auc])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Model: "model"
__________________________________________________________________________________________________
Layer (type)  

In [46]:
filepath="weights_3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')

model.fit([text_train,cat_num_train_feat], y_train, epochs=20, verbose=1, batch_size=250, validation_data=([text_cv,cat_num_cv_feat], y_cv), callbacks=[tensorboard,checkpoint])

Train on 53531 samples, validate on 22942 samples
Epoch 1/20
53500/53531 [============================>.] - ETA: 0s - loss: 0.5175 - auc: 0.5172
Epoch 00001: val_auc improved from -inf to 0.55825, saving model to weights_3.hdf5
53531/53531 [==============================] - 140s 3ms/sample - loss: 0.5176 - auc: 0.5165 - val_loss: 0.4840 - val_auc: 0.5583
Epoch 2/20
53500/53531 [============================>.] - ETA: 0s - loss: 0.4858 - auc: 0.5234
Epoch 00002: val_auc did not improve from 0.55825
53531/53531 [==============================] - 40s 747us/sample - loss: 0.4857 - auc: 0.5225 - val_loss: 0.4858 - val_auc: 0.5080
Epoch 3/20
53500/53531 [============================>.] - ETA: 0s - loss: 0.4657 - auc: 0.5816
Epoch 00003: val_auc improved from 0.55825 to 0.63197, saving model to weights_3.hdf5
53531/53531 [==============================] - 41s 768us/sample - loss: 0.4656 - auc: 0.5826 - val_loss: 0.5069 - val_auc: 0.6320
Epoch 4/20
53500/53531 [============================>.] -

#### compiling model weights

In [47]:
from tensorflow.keras.optimizers import Adam
model.load_weights("weights_3.hdf5")
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[auc])

#### Model visualization

In [48]:
print("Train AUC",roc_auc_score(y_train,(model.predict([text_train,cat_num_train_feat]))))
print("-"*50)
print("Cv AUC",roc_auc_score(y_cv,model.predict([text_cv,cat_num_cv_feat])))
print("-"*50)
print("Test AUC",roc_auc_score(y_test,model.predict([text_test,cat_num_test_feat])))

Train AUC 0.8204224648514182
--------------------------------------------------
Cv AUC 0.751834165899868
--------------------------------------------------
Test AUC 0.7627339352050855


<img src="model_3.png">

In [76]:
from prettytable import PrettyTable
    
x = PrettyTable(["Model", "Train AUC", "Cv AUC", "Test AUC"])

x.add_row(["Model 1", 0.78,0.74,0.76])
x.add_row(["Model 2", 0.82,0.74,0.75])
x.add_row(["Model 3", 0.82,0.75,0.76])

print(x.get_string(title="Model results"))

+---------+-----------+--------+----------+
|  Model  | Train AUC | Cv AUC | Test AUC |
+---------+-----------+--------+----------+
| Model 1 |    0.78   |  0.74  |   0.76   |
| Model 2 |    0.82   |  0.74  |   0.75   |
| Model 3 |    0.82   |  0.75  |   0.76   |
+---------+-----------+--------+----------+
